In [1]:
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from decimal import Decimal
from benchmark.alphaxutils import *

import requests
import pandas as pd

In [2]:
r = requests.get('https://tokens.coingecko.com/uniswap/all.json')
cgtokens = pd.DataFrame(eval(r.text)['tokens'])

tokenDecimals = {}
tokenDecimals['ETH'] = '18'
for i in cgtokens.index:
    tokenDecimals[cgtokens.symbol[i]] = str(cgtokens.decimals[i])

In [3]:
threshold_orders = 6
support_partial = True

# # ETH side orders
# tkn_y = 'ETH'
# tkn_x = 'USDC'

# realisticByTarget_inputAmount = 10 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
# realisticBySource_inputAmount = 30000 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

# order_params = {
#     'liquidity':'5',
#     'highestRate':'0.0005', # 2000
#     'lowestRate':'0.0004',  # 2500
#     'marginalRate':'0.0005',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params2 = {
#     'liquidity':'4',
#     'highestRate':'0.0006', # 1666
#     'lowestRate':'0.0005',  # 2000
#     'marginalRate':'0.0006',
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }
# order_params3 = {
#     'liquidity':Decimal('10'),
#     'highestRate':Decimal('0.0006'), # 1666
#     'lowestRate':Decimal('0.0005'),  # 2000
#     'marginalRate':Decimal('0.0006'),
#     'dec_y': tokenDecimals[tkn_y],
#     'dec_x': tokenDecimals[tkn_x],
#     }


# USDC side orders
tkn_y = 'USDC'
tkn_x = 'ETH'

realisticByTarget_inputAmount = 30000 * 10** int(tokenDecimals[tkn_y])  # by Target = USDC amount
realisticBySource_inputAmount = 10 * 10** int(tokenDecimals[tkn_x])  # by Source = ETH amount

order_params = {
    'liquidity':'20000',
    'highestRate':'2500',
    'lowestRate':'2000',
    'marginalRate':'2500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params2 = {
    'liquidity':'17000',
    'highestRate':'3500',
    'lowestRate':'3000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }
order_params3 = {
    'liquidity':'15000',
    'highestRate':'3500',
    'lowestRate':'2000',
    'marginalRate':'3500',
    'dec_y': tokenDecimals[tkn_y],
    'dec_x': tokenDecimals[tkn_x],
    }

orders = []
for i in range(1):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]
    orders += [Order(order_params3)]

In [4]:
print("inputAmount",realisticByTarget_inputAmount)
TargetActions2 = mpr_matchByTarget(realisticByTarget_inputAmount, orders, threshold_orders, support_partial)
total_output_Target2 = sum([TargetActions2[k]['dx'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_input_Target2 = sum([TargetActions2[k]['dy_specified'] for k in TargetActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Target2)
print("Total Return Real:", total_output_Target2)
print("Rate Real:",(total_input_Target2/total_output_Target2))
TargetActions2

inputAmount 30000000000
Input Amount Real: 29999.999999
Total Return Real: 9.877286043821534098
Rate Real: 3037.271560821677202884192232184135009572082317288518156108707059630571251222815926492444147272226122


{0: {'dy_specified': 2733970306, 'dx': 1109601499190347106},
 1: {'dy_specified': 17000000000, 'dx': 5246313899181075013},
 2: {'dy_specified': 10266029693, 'dx': 3521370645450111979}}

In [5]:
print("inputAmount",realisticBySource_inputAmount)
SourceActions2 = mpr_matchBySource(realisticBySource_inputAmount, orders, threshold_orders, support_partial)
total_input_Source2 = sum([SourceActions2[k]['dx_specified'] for k in SourceActions2.keys()])  / Decimal('10')**Decimal(tokenDecimals[tkn_x])
total_output_Source2 = sum([SourceActions2[k]['dy'] for k in SourceActions2.keys()]) / Decimal('10')**Decimal(tokenDecimals[tkn_y])
print("Input Amount Real:", total_input_Source2)
print("Total Return:", total_output_Source2)
print("Rate:",total_output_Source2/total_input_Source2)
SourceActions2

inputAmount 10000000000000000000
Input Amount Real: 9.999999999999999999
Total Return: 30297.621162
Rate: 3029.762116200000000302976211620000000030297621162000000003029762116200000000302976211620000000030298


{0: {'dx_specified': 1205909808787015248, 'dy': 2967549208},
 1: {'dx_specified': 5246313899181075013, 'dy': 17000000000},
 2: {'dx_specified': 3547776292031909738, 'dy': 10330071954}}

In [9]:
from benchmark import util
from alpha_x_router_GenII import mpr_matchBySource
from alpha_x_router_GenII import mpr_matchByTarget

from benchmark.alphaxutils import Order

for fileName in ['ArbitraryMatch', 'BigPoolMatch', 'EthUsdcMatch']:
    for test in util.read('', f'benchmark/resources/match/{fileName}'):
        if any(order_params['lowestRate'] == order_params['highestRate'] for order_params in test['orders']):
            continue
        method = globals()['mpr_' + test['method'].split('Amount')[0]]
        amount = int(test['amount'])
        orders = [Order({**order_params, 'dec_y': 0, 'dec_x': 0}) for order_params in test['orders']]
        oldActions = test['actions']
        newActions = method(amount, orders, len(oldActions), True)
        if test['method'] == 'matchBySourceAmount':
            oldInputSum  = sum(int(action['input'       ]) for action in oldActions)
            newInputSum  = sum(int(action['dx_specified']) for action in newActions.values())
            oldOutputSum = sum(int(action['output'      ]) for action in oldActions)
            newOutputSum = sum(int(action['dy'          ]) for action in newActions.values())
            if not (newInputSum <= oldInputSum and newOutputSum >= oldOutputSum):
                print(util.dumps(test, indent = 4))
                for key in newActions:
                    obj = {
                        'id': key,
                        'input': str(newActions[key]['dx_specified']),
                        'output': str(newActions[key]['dy']),
                    }
                    print(util.dumps(obj, indent = 4))
                print('oldInputSum  =', oldInputSum )
                print('newInputSum  =', newInputSum )
                print('oldOutputSum =', oldOutputSum)
                print('newOutputSum =', newOutputSum)
                assert False
        if test['method'] == 'matchByTargetAmount':
            oldInputSum  = sum(int(action['input'       ]) for action in oldActions)
            newInputSum  = sum(int(action['dx'          ]) for action in newActions.values())
            oldOutputSum = sum(int(action['output'      ]) for action in oldActions)
            newOutputSum = sum(int(action['dy_specified']) for action in newActions.values())
            if not (newInputSum >= oldInputSum and newOutputSum <= oldOutputSum):
                print(util.dumps(test, indent = 4))
                for key in newActions:
                    obj = {
                        'id': key,
                        'input': str(newActions[key]['dx']),
                        'output': str(newActions[key]['dy_specified']),
                    }
                    print(util.dumps(obj, indent = 4))
                print('oldInputSum  =', oldInputSum )
                print('newInputSum  =', newInputSum )
                print('oldOutputSum =', oldOutputSum)
                print('newOutputSum =', newOutputSum)
                assert False


** Partial Match (90.28427%) **
** Partial Match (64.40659%) **
** Partial Match (48.22022%) **
** Partial Match (37.41799%) **
** Partial Match (29.84629%) **
** Partial Match (24.33042%) **
** Partial Match (36.73077%) **
** Partial Match (18.31264%) **
** Partial Match (10.92457%) **
** Partial Match (7.21332%) **
** Partial Match (5.07797%) **
** Partial Match (3.73064%) **
** Partial Match (2.82119%) **
** Partial Match (2.17452%) **
** Partial Match (1.69514%) **
** Partial Match (90.28428%) **
** Partial Match (64.40660%) **
** Partial Match (48.22023%) **
** Partial Match (37.41800%) **
** Partial Match (29.84630%) **
** Partial Match (24.33043%) **
** Partial Match (37.31648%) **
** Partial Match (18.60550%) **
** Partial Match (11.10028%) **
** Partial Match (7.33046%) **
** Partial Match (5.16163%) **
** Partial Match (3.79338%) **
** Partial Match (2.86998%) **
** Partial Match (2.21354%) **
** Partial Match (1.72705%) **
** Partial Match (90.28426%) **
** Partial Match (64

AssertionError: 